<a href="https://colab.research.google.com/github/namaws/Income-analysis-and-prediction/blob/main/Data_analysis_and_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Intro to urban computing - Assignment 3
### Data Analysis


In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/MyDrive/SMOD/new version')